In [8]:
# Try the PyPI version instead of GitHub
%pip install -q --no-cache-dir pyterrier-t5

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

/home/killdollar/btp/Information-Retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
indexer = pt.IterDictIndexer('./indices/msmarco-passage')
index = indexer.path

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [10]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=100)

/tmp/ipykernel_5110/302786154.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=100)


In [11]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

In [12]:
# Process each query manually to avoid the document ID mismatch
import pandas as pd
topics = dataset.get_topics()
all_results = []

for idx, row in topics.iterrows():
    query_id = row['qid']
    query_text = row['query']
    
    print(f"Processing query {idx+1}/{len(topics)}: {query_text}")
    
    # Step 1: Get initial results from BM25
    initial_results = bm25.search(query_text)
    
    # Step 2: Fetch document text from the dataset directly
    docnos = initial_results['docno'].tolist()
    doc_texts = {}
    
    # Get document text from dataset safely
    for docno in docnos:
        try:
            # Convert to string if needed
            doc_id = str(docno)
            text = dataset.get_document_text(doc_id)
            doc_texts[docno] = text
        except Exception as e:
            # If document not found, use a placeholder
            doc_texts[docno] = "Document text not available"
    
    # Add text to results
    initial_results['text'] = initial_results['docno'].map(doc_texts)
    
    # Step 3: Apply MonoT5 reranking
    # Check if MonoT5 is working (otherwise revert to sorting by BM25 score)
    try:
        reranked_results = monoT5.transform(initial_results)
        # Sort by MonoT5 score and take top 10
        top_results = reranked_results.sort_values('score', ascending=False).head(10)
    except Exception as e:
        print(f"MonoT5 reranking error: {e}")
        # Fallback to BM25 ranking
        top_results = initial_results.head(10)
    
    # Add query ID to results
    top_results['qid'] = query_id
    
    # Store results
    all_results.append(top_results)
    
    # Show progress
    if idx == 0:
        print("\nSample results for first query:")
        print(top_results[['docno', 'score']].head(3))

# Combine all results
combined_results = pd.concat(all_results) if all_results else pd.DataFrame()

# Save results to CSV
combined_results.to_csv('monot5_results.csv', index=False)

print(f"\nRetrieved {len(combined_results)} documents across {len(topics)} queries")
print(f"Results saved to monot5_results.csv")

# Show summary of results
print("\nTop documents for first few queries:")
print(combined_results.groupby('qid').head(1)[['qid', 'docno', 'score']].head(5))

Processing query 1/43: do goldfish grow


monoT5: 100%|██████████| 25/25 [00:06<00:00,  4.11batches/s]



Sample results for first query:
     docno      score
0  8182161 -13.589843
1  6139386 -13.589843
2  3288600 -13.589843
Processing query 2/43: what is wifi vs bluetooth


monoT5: 100%|██████████| 25/25 [00:06<00:00,  4.08batches/s]


Processing query 3/43: why did the us volunterilay enter ww1


...

Processing query 42/43: hydrogen is a liquid below what temperature


monoT5: 100%|██████████| 25/25 [00:05<00:00,  4.42batches/s]


Processing query 43/43: difference between a mcdouble and a double cheeseburger


monoT5: 100%|██████████| 25/25 [00:05<00:00,  4.54batches/s]


Retrieved 425 documents across 43 queries
Results saved to monot5_results.csv

Top documents for first few queries:
       qid    docno      score
0   156493  8182161 -13.589843
0  1110199  3838645 -13.018057
0  1063750  4788295 -13.544714
0   130510  1494936 -11.937598
0   489204  1479542 -13.447414
